In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/submission-7/submission_7.csv
/kaggle/input/submission-10/submission_10.csv
/kaggle/input/frauddetect/sample_submission.csv
/kaggle/input/frauddetect/test_identity.csv
/kaggle/input/frauddetect/train_identity.csv
/kaggle/input/frauddetect/test_transaction.csv
/kaggle/input/frauddetect/train_transaction.csv


In [3]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import gc
from hyperopt import hp, fmin, tpe, Trials
from sklearn.metrics import roc_auc_score
from functools import partial
from hyperopt.pyll.base import scope
import xgboost as xgb
from xgboost import cv
from sklearn.metrics import roc_auc_score
from xgboost import plot_importance
import matplotlib.pyplot as plt

In [4]:
drop_vcd = ['V2', 'V5', 'V7', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V17', 'V18', 'V19', 'V21', 'V22', 'V23', 'V25', 'V27', 'V28', 'V29', 'V31', 'V32', 'V33', 'V34', 'V35', 'V37', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V46', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V55', 'V57', 'V58', 'V59', 'V60', 'V61', 'V63', 'V64', 'V66', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V77', 'V79', 'V81', 'V83', 'V84', 'V85', 'V87', 'V88', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'V96', 'V97', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106', 'V109', 'V110', 'V112', 'V113', 'V114', 'V116', 'V117', 'V119', 'V122', 'V124', 'V125', 'V126', 'V128', 'V129', 'V132', 'V133', 'V134', 'V135', 'V137', 'V140', 'V141', 'V144', 'V145', 'V146', 'V148', 'V149', 'V150', 'V151', 'V152', 'V153', 'V154', 'V155', 'V157', 'V158', 'V159', 'V161', 'V163', 'V164', 'V167', 'V168', 'V170', 'V172', 'V176', 'V177', 'V178', 'V179', 'V181', 'V182', 'V183', 'V184', 'V186', 'V187', 'V189', 'V190', 'V191', 'V192', 'V193', 'V194', 'V195', 'V196', 'V197', 'V199', 'V200', 'V202', 'V204', 'V206', 'V208', 'V211', 'V212', 'V213', 'V214', 'V216', 'V217', 'V218', 'V219', 'V221', 'V224', 'V225', 'V227', 'V228', 'V229', 'V230', 'V231', 'V232', 'V233', 'V235', 'V236', 'V237', 'V239', 'V241', 'V242', 'V243', 'V245', 'V246', 'V247', 'V248', 'V249', 'V250', 'V252', 'V253', 'V254', 'V255', 'V256', 'V257', 'V258', 'V259', 'V262', 'V263', 'V265', 'V266', 'V269', 'V270', 'V272', 'V273', 'V275', 'V276', 'V277', 'V279', 'V280', 'V287', 'V288', 'V290', 'V292', 'V293', 'V294', 'V295', 'V296', 'V297', 'V298', 'V299', 'V300', 'V302', 'V304', 'V306', 'V308', 'V311', 'V313', 'V315', 'V316', 'V317', 'V318', 'V319', 'V321', 'V322', 'V323', 'V324', 'V327', 'V328', 'V329', 'V330', 'V331', 'V333', 'V334', 'V335', 'V337', 'V338', 'C1', 'C2', 'C4', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C14', 'D2', 'D3', 'D4', 'D7', 'D12']

In [5]:
def preprocess_txn( txn ):
    # drop v, c, d columns
    
    txn = txn.drop( drop_vcd, axis = 1 )
    
    # addr1 & addr2
    
    addr1_retain_list = [299.0, 325.0, 204.0, 264.0, 330.0, 315.0, 441.0, 272.0, 123.0, 126.0, 184.0, 337.0, 191.0, 181.0, 143.0, 476.0, 310.0, 327.0, 472.0]
    txn.loc[txn.addr1.isin(addr1_retain_list) == False, 'addr1'] = "Others"
    txn.addr1.fillna('Others', inplace = True)
    txn.addr1 = txn.addr1.astype(str)
    
    addr2_retain_list = [87.0, 60.0, 96.0, 32.0, 65.0]
    txn.loc[txn.addr2.isin(addr2_retain_list) == False, 'addr2'] = "Others"
    txn.addr2.fillna('Others', inplace = True)
    txn.addr2 = txn.addr2.astype(str)
    
    # P_emaildomain
    
    txn.loc[ txn['P_emaildomain'].isin(['gmail', 'gmail.com']),'P_emaildomain'] = 'Google'
    txn.loc[ txn['P_emaildomain'].isin(['yahoo.fr', 'yahoo.es', 'yahoo.de', 'yahoo.com.mx', 'yahoo.com', 'yahoo.co.uk', 'yahoo.co.jp']),'P_emaildomain'] = 'Yahoo'
    txn.loc[ txn['P_emaildomain'].isin(['hotmail.com','outlook.com','msn.com', 'live.com.mx', 'hotmail.es','hotmail.co.uk', 'hotmail.de','outlook.es', 'live.com', 'live.fr','hotmail.fr']), 'P_emaildomain'] = 'Microsoft'
    txn.loc[ txn['P_emaildomain'].isin(['anonymous.com']), 'P_emaildomain'] = 'Anon'
    
    p_domain_retain_list = ['protonmail.com', 'mail.com', 'aim.com', 'embarqmail.com', 'mac.com', 'cableone.net', 'icloud.com', 'bellsouth.net', 'comcast.net',
                            'suddenlink.net', 'charter.net', 'frontier.com', 'frontiernet.net', 'ymail.com', 'aol.com',
                            'Google', 'Microsoft', 'Anon', 'Yahoo']
    
    txn.loc[txn['P_emaildomain'].isin(p_domain_retain_list) == False, 'P_emaildomain'] = 'Other'
    txn.P_emaildomain.fillna('Unknown', inplace = True)
    
    # R_emaildomain
    txn.loc[ txn['R_emaildomain'].isin(['gmail', 'gmail.com']),'R_emaildomain'] = 'Google'
    txn.loc[ txn['R_emaildomain'].isin(['yahoo.fr', 'yahoo.es', 'yahoo.de', 'yahoo.com.mx', 'yahoo.com', 'yahoo.co.uk', 'yahoo.co.jp']),'R_emaildomain'] = 'Yahoo'
    txn.loc[ txn['R_emaildomain'].isin(['hotmail.com','outlook.com','msn.com', 'live.com.mx', 'hotmail.es','hotmail.co.uk', 'hotmail.de','outlook.es', 'live.com', 'live.fr','hotmail.fr']), 'R_emaildomain'] = 'Microsoft'
    
    r_domain_retain_list = ['protonmail.com', 'mail.com', 'netzero.net', 'icloud.com', 'Google', 'Microsoft', 'suddenlink.net', 'charter.net', 'ymail.com', 'rocketmail.com', 'aim.com', 'Yahoo', 'earthlink.net']
    
    txn.loc[txn['R_emaildomain'].isin(r_domain_retain_list) == False, 'R_emaildomain'] = 'Other'
    txn.R_emaildomain.fillna('Unknown', inplace=True)
    
    # card1-6
    card = ["card" + str(i) for i in range(1, 7)]
    card_encode = { 'american express' : 3, 'discover' : 4, 'mastercard' : 2, 'visa' : 1, 'credit' : 0, 'debit' : 1, 'charge card' : 3, 'debit or credit' : 4}
    txn[card] = txn[card].replace(card_encode)
    
    # m1-9
    m = ['M' + str(i) for i in range(1,10)]
    m_encode = {'T' : 1, 'F' : 0, 'M0' : 0 , 'M1' : 1, 'M2' : 2}
    txn[m] = txn[m].replace(m_encode)
    
    
    #TransactionDT
    txn['TransactionDT'] = txn['TransactionDT']//86400
    
#     txn['TransactionDay'] = txn.TransactionDT//(86400)
#     txn['TransactionTime'] = (txn.TransactionDT/(86400)) - (txn.TransactionDT//(86400))
#     txn = txn.drop('TransactionDT', axis = 1)
#     txn['card1'] = txn['card1']//100
    
    return txn

In [6]:
import re

#id_31
def browser_version(lst):
    
    r = re.compile('[0-9][0-9]|[0-9]\.[0-9]')
    final = []
    
    browser = ""
    version = ""
    browser_list = ['firefox', 'chrome', 'ie', 'edge', 'safari', 'samsung browser', 'facebook', 'opera']
    
    for i in lst:
        
        if pd.isnull(i):
            final.append('misc')
        else:
            for br in browser_list:
                if br in i.lower():
                    browser = br
                    break
            ver = r.search(i)
            if ver:
                version = ver.group()
            else:
                version = 'generic'
    
            final.append( str(browser) )
    
#     valid = pd.DataFrame( [final, lst], index = ['new', 'old']).T
#     display(final)
    return final

def device_info(dinfo):
    brand_count = []

    for i in dinfo:
        if str(i) == 'nan':
            brand_count.append('other')
        elif 'hisense' in i.lower():
            brand_count.append('hisense')
        elif 'sm' in i.lower() or 'samsung' in i.lower() or i.lower()[:2] in ['gt', 'sc', 'sg', 'sg', 'sp']:
            brand_count.append('samsung')
        elif 'moto' in i.lower() or 'xt' in i.lower():
            brand_count.append('moto')
        elif 'lg' in i.lower() or i.lower()[:2] in ['vs']:
            brand_count.append('lg')
        elif 'trident' in i.lower():
            brand_count.append('trident')
        elif 'huawei' in i.lower() or i.lower()[:2] in ['hi']:
            brand_count.append('huawei')
        elif 'lenovo' in i.lower():
            brand_count.append('lenovo')
        elif 'rv' in i.lower():
            brand_count.append('rv')
        elif 'mac' in i.lower() or 'ios' in i.lower():
            brand_count.append('apple')
        elif 'htc' in i.lower():
            brand_count.append('htc')
        elif 'asus' in i.lower():
            brand_count.append('asus')
        elif 'windows' in i.lower() or 'rv:11.0' in i.lower():
            brand_count.append('windows')
        elif 'blade' in i.lower():
            brand_count.append('blade')
        elif 'mi' in i.lower():
            brand_count.append('xiaomi')
        elif 'linux' in i.lower():
            brand_count.append('linux')
        elif 'sony' in i.lower() or i.lower()[:2] in ['d5', 'd6', 'e2', 'e5', 'e6', 'f3', 'f5', 'f8', 'g3', 'g6', 'g8', 'h3', 'h5']:
            brand_count.append('sony')
        elif 'pixel' in i.lower() or 'nexus' in i.lower():
            brand_count.append('google')
        elif i.lower()[:2] in ['50', '60', '70', '80', '90'] or 'alcatel' in i.lower() or i.lower()[:4] in ['one ']:
            brand_count.append('alcatel')
        elif 'ilium' in i.lower():
            brand_count.append('ilium')
        elif i.lower()[:2] in ['kf']:
            brand_count.append('kindle')
        elif i.lower()[:2] in ['ta']:
            brand_count.append('nokia')
        elif i.lower()[:2] in ['z4', 'z5', 'z7', 'z8', 'z9'] or 'zte' in i.lower():
            brand_count.append('zte')
        else:
            brand_count.append('other')
            
    return brand_count

#id_33
def resolutions( res ):
    
    def bins():
        resolutions = {
            'NA': -10,
            'SD' : 0,
            'HD' : 1280*720,
            'FHD' : 1920*1080,
            'QHD' : 2560*1440,
            '4K' : 3840*2160,
            '5K' : 5120*2880,
        }

        bins = list(resolutions.values())
        bins += [150000000]
        labels = list(resolutions.keys())
        
        return bins, labels
    
    def parse_res( res ):
        
        if pd.isnull(res):
            return -1
        mul = res.find('x')
        h = res[0:mul]
        w = res[mul+1:]
        
        return ( int(w) * int(h) )

    res = [parse_res(i) for i in res]

    bins, labels = bins()
    res = pd.cut( res, bins = bins, labels = labels)
    res = res.astype('object')
    return res

#id13
def bin13(id13):
    
    id13 = id13.fillna(-1)
    
    bins = [-10, 0, 25, 50, 100]
    labels = [0, 1, 2, 3]
    
    id13 = pd.cut( id13, bins = bins, labels = labels )
    id13 = id13.astype('float64')
    return id13

#id34
def parse34(s):
    if not pd.isnull(s):
        return s[13:]
    return s


def preprocess_id(dd):
    
    # >90% Null Values
    # drop_list = [ 'id_18', 'id_21', 'id_22', 'id_23', 'id_26', 'id_27', 'id_07', 'id_08', 'id_25', 'id_24' ]
    drop_list = [ 'id_18', 'id_21', 'id_22', 'id_23', 'id_26', 'id_27', 'id_07', 'id_08', 'id_25', 'id_24', 'id_30',
                 'id_32', 'id_03', 'id_04', 'id_09', "id_10" ]
    dd = dd.drop(drop_list, axis = 1)
    
    #Device Type
    lol = dd[['id_31', 'DeviceType']]
    for index, row in lol.iterrows():
        if not pd.isnull(row.id_31) and pd.isnull(row.DeviceType):
            if row.id_31 == 'ie 11.0 for desktop':
                row['DeviceType'] = 'desktop'
    
    #id_31
    lol.id_31 = list( browser_version( list(dd.id_31) ) )
    dd[['id_31', 'DeviceType']] = lol[['id_31', 'DeviceType']]
    dd.DeviceType = dd.DeviceType.fillna('desktop')
    
    #Device Info
    brand_count = device_info( list(dd.DeviceInfo) )
    dd.DeviceInfo = brand_count
    
    #categorical
    #id_14
    dd.id_14 = dd.id_14/60
    
    #categorical
    #id34
    dd.id_34 = dd.id_34.apply( lambda x : parse34(x) )
    
    #id_33
    res = dd.id_33
    res = resolutions(res)
    dd.id_33 = res
    
    #id_13
    id_13 = dd.id_13
    id_13 = bin13(id_13)
    dd.id_13 = id_13
    
    #id_16
    dd.id_16 = dd.id_16.fillna('NotFound')
        
    #id_15
    dd.id_15 = dd.id_15.fillna('Unknown')
    
    #id_35,36,37,38
    dd.id_35 = dd.id_35.fillna('F')
    dd.id_36 = dd.id_36.fillna('F')
    dd.id_37 = dd.id_37.fillna('T')
    dd.id_38 = dd.id_38.fillna('F')
    
    #id_28, 29
    dd.id_28 = dd.id_28.fillna('New')
    dd.id_29 = dd.id_29.fillna('NotFound')
    
    #id_02
    id2 = dd.id_02
    id2 = id2.fillna(id2.mean())
    dd.id_02 = id2
    
    #id_05, id_06
    dd.id_05 = dd.id_05.fillna(0)
    dd.id_06 = dd.id_06.fillna(0)
    
    #id_11
    dd.id_11 = dd.id_11.fillna(100.0)
    
    #id_17, id_19, id_20
    dd.id_17 = dd.id_17.fillna( dd.id_17.mean() )
    dd.id_19 = dd.id_19.fillna( dd.id_19.mean() )
    dd.id_20 = dd.id_20.fillna( dd.id_20.mean() )
    
    return dd

In [7]:
def submit(y_pred, num):
    sample = pd.read_csv('../input/frauddetect/sample_submission.csv')
    sample.isFraud = y_pred
    sub_name = 'submission_' + str(num) + '.csv'
    sample.to_csv(sub_name, index=False)
    
def get_scale_pos_weight(y):
    return (y.value_counts().iloc[0])/(y.value_counts().iloc[1])

In [8]:
def proc():
    tt = pd.read_csv('../input/frauddetect/train_transaction.csv', index_col='TransactionID')
    ti = pd.read_csv('../input/frauddetect/train_identity.csv', index_col='TransactionID')
    
    tt = preprocess_txn(tt)
    ti = preprocess_id(ti)
    
    t1 = pd.merge(tt, ti, left_index=True, right_index=True, how='left')
    obj = t1.dtypes[ t1.dtypes == 'object' ].index.tolist()
    t1[obj] = t1[obj].fillna('nullval')
    
    x = t1.sort_values('TransactionDT').drop('isFraud', axis = 1)
    y = pd.DataFrame( t1.sort_values('TransactionDT').isFraud )
    
    del tt, ti, t1
    
    gc.collect()
    
    obj_list = x.dtypes[x.dtypes=='object'].index.tolist()
    obj_list += x.dtypes[x.dtypes=='category'].index.tolist()

    columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), obj_list)], remainder='passthrough')
    
    x = x.fillna(-999)

    x = np.array(columnTransformer.fit_transform(x))
    
    test_txn = pd.read_csv('../input/frauddetect/test_transaction.csv', index_col = 'TransactionID')
    test_id = pd.read_csv('../input/frauddetect/test_identity.csv', index_col = 'TransactionID')
    
    test_txn = preprocess_txn(test_txn)
    test_id = preprocess_id(test_id)
    
    test = pd.merge(test_txn, test_id, left_index=True, right_index=True, how='left')
    
    test[obj] = test[obj].fillna('nullval')
    
    test = test.fillna(-999)
    
    test = pd.DataFrame( np.array(columnTransformer.transform(test)) )

    features = columnTransformer.get_feature_names()
    
    x = pd.DataFrame(x, columns = features)
    test.columns = features
    
    display( x.head() )
    display( y.head() )
    display( test.head() )
    
#     idxT = x.index[:3*len(x)//4]
#     idxV = x.index[3*len(x)//4:]
#     idyT = y.index[:3*len(y)//4]
#     idyV = y.index[3*len(y)//4:]

#     x_train = x.loc[idxT]
#     y_train = y.loc[idyT]
#     x_val = x.loc[idxV]
#     y_val = y.loc[idyV]

    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.25, random_state=42, stratify=y)
    
    data = {
        'x_train' : x_train,
        'y_train' : y_train,
        'x_val' : x_val,
        'y_val' : y_val,
        'test' : test,
        'x' : x,
        'y' : y
    }
    return data

In [8]:
data = proc()

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,encoder__x0_C,encoder__x0_H,encoder__x0_R,encoder__x0_S,encoder__x0_W,encoder__x1_123.0,encoder__x1_126.0,encoder__x1_143.0,encoder__x1_181.0,encoder__x1_184.0,...,id_01,id_02,id_05,id_06,id_11,id_13,id_14,id_17,id_19,id_20
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,22608.0,1.0,-11.0,100.0,0.0,-6.0,166.0,410.0,146.0


,isFraud
TransactionID,
2987002,0
2990409,0
2990410,0
2990411,0
2990412,0


,encoder__x0_C,encoder__x0_H,encoder__x0_R,encoder__x0_S,encoder__x0_W,encoder__x1_123.0,encoder__x1_126.0,encoder__x1_143.0,encoder__x1_181.0,encoder__x1_184.0,...,id_01,id_02,id_05,id_06,id_11,id_13,id_14,id_17,id_19,id_20
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,70787.0,0.0,0.0,100.0,0.0,-8.0,166.0,542.0,144.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [12]:
tt = pd.read_csv('../input/frauddetect/train_transaction.csv', index_col='TransactionID')
ti = pd.read_csv('../input/frauddetect/train_identity.csv', index_col='TransactionID')

In [14]:
ti.id_14.value_counts()

-300.0    26582
-360.0    10135
-480.0     7726
-420.0     2805
-600.0      296
 60.0       232
 0.0        128
-240.0      100
-180.0       70
-540.0       66
 480.0       50
 540.0       38
 600.0       33
 180.0       32
 120.0       27
 420.0       10
 240.0        9
 330.0        9
 270.0        8
 300.0        8
 720.0        7
-210.0        1
 360.0        1
-120.0        1
Name: id_14, dtype: int64

In [11]:
preprocess_id(ti)

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,id_01,id_02,id_05,id_06,id_11,id_12,id_13,id_14,id_15,id_16,...,id_29,id_31,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987008,-5.0,98945.0,0.0,-5.0,100.0,NotFound,2.0,-5.0,New,NotFound,...,NotFound,safari,HD,1,T,F,F,T,mobile,apple
2987010,-5.0,191631.0,0.0,0.0,100.0,NotFound,3.0,NaN,Found,Found,...,Found,chrome,NA,NaN,F,F,T,T,desktop,windows
2987016,0.0,7460.0,1.0,0.0,100.0,NotFound,0.0,-5.0,Found,Found,...,Found,chrome,HD,2,T,F,T,T,desktop,apple
2987040,-10.0,116098.0,0.0,0.0,100.0,NotFound,3.0,NaN,Found,Found,...,Found,chrome,NA,NaN,F,F,T,T,desktop,windows
2987048,-5.0,257037.0,0.0,0.0,100.0,NotFound,3.0,NaN,New,NotFound,...,NotFound,chrome,NA,NaN,F,F,T,T,desktop,windows
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3577472,-5.0,345613.0,-11.0,-30.0,100.0,NotFound,2.0,NaN,New,NotFound,...,NotFound,chrome,NA,NaN,F,F,T,F,mobile,samsung
3577499,-5.0,89753.0,0.0,0.0,100.0,NotFound,3.0,NaN,Found,Found,...,Found,firefox,NA,NaN,F,F,T,F,desktop,windows
3577521,-15.0,145955.0,0.0,0.0,100.0,NotFound,2.0,NaN,Found,Found,...,Found,chrome,NA,NaN,F,F,T,F,mobile,sony


In [ ]:
drop_list = [ 'id_18', 'id_21', 'id_22', 'id_23', 'id_26', 'id_27', 'id_07', 'id_08', 'id_25', 'id_24', 'id_30',
                 'id_14', 'id_34', 'id_32', 'id_03', 'id_04', 'id_09', "id_10" ]

In [ ]:
for i in drop_list:
    print( ti[i].value_counts(dropna=0, normalize=True) )

In [ ]:
tt.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
tt2 = tt
tt2['TransactionDT'] = tt2['TransactionDT']//86400
tt2['TransactionDT']

In [ ]:
(tt2.card1//100).value_counts()

In [ ]:
tt2['card1'] = tt2['card1']//100

In [ ]:
fig, ax = plt.subplots(figsize=(60,20))
sns.scatterplot( x='TransactionDT', y='card1', hue='isFraud', data=tt2, ax=ax , size='isFraud', sizes=(150, 50))

In [ ]:
# params = {
#     'objective':'binary:logistic',
#     'max_depth': 8,
#     'alpha': 10,
#     'learning_rate': 0.1,
#     'eval_metric': 'auc'
# }

# dmat = xgb.DMatrix( data = X, label = y)

# xgb_cv = cv(dtrain = dmat, params = params, nfold = 3,
#             num_boost_round=50, metrics="auc", as_pandas=True, seed = 42)

In [ ]:
# param_space = {
#     'max_depth' : [4, 8, 16],
#     'alpha' : [10],
#     'learning_rate' : [0.1],
#     'num_boost_rounds' : [50, 100],
#     'scale_pos_weight' : [scale_pos_weight, 50]
# }

# lr = 0.1
# a = 10

# count = 1
# for d in param_space['max_depth']:
#     for w in param_space['scale_pos_weight']:
#         for b in param_space['num_boost_rounds']:

#             print(f'ROUND {count}\n')
#             print(f'learning_rate = {lr}, depth = {d}, alpha = {a}, boost_rounds = {b}, scale_pos_weight = {w}\n')
#             count += 1

#             params = {
#                 'objective':'binary:logistic',
#                 'max_depth': d,
#                 'alpha': a,
#                 'learning_rate': lr,
#                 'eval_metric': 'auc',
#                 'scale_pos_weight' : w
#             }

#             xgb_cv = cv(dtrain = dmat, params = params, nfold = 3,
#                         num_boost_round=b, metrics="auc", as_pandas=True, seed = 42)

#             print(f'train_auc_mean = {round(xgb_cv.iloc[-1, 0],4)}\ntest_auc_mean = {round(xgb_cv.iloc[-1, 2], 4)}\n')

## HyperOpt Begins

In [ ]:
def optimize(params, x, y):
    model = xgb.XGBClassifier(
        objective='binary:logistic',
        missing = -999,
        eval_metric='auc',
        random_state = 42,
        scale_pos_weight = scale_pos_weight,
        verbosity=1,
        tree_method='gpu_hist',
        gpu_id=0,
        **params)
    
    x_train = x.loc[idxT]
    y_train = y.loc[idyT]
    
    x_val = x.loc[idxV]
    y_val = y.loc[idyV]
    
    model.fit(x_train, y_train)
    y_pred = model.predict_proba(x_val)
    score = roc_auc_score(y_val, y_pred[:, 1])
    
    del x_train, y_train, x_val, y_val, y_pred
    gc.collect()
    
    return -(score)


param_space = {
    'max_depth' : scope.int(hp.quniform("max_depth", 4, 16, 2)),
    'n_estimators' : scope.int(hp.quniform("n_estimators", 500, 1000, 1)),
    'learning_rate' : hp.uniform("learning_rate", 0.1, 0.2),
    'alpha' : hp.quniform('alpha', 5, 16, 1),
}

trial = Trials()

optimization_function = partial( optimize, x=X, y=y)

#     'max_depth': 8,
#     'alpha': 10,
#     'learning_rate': 0.1,
#     'n_estimators': 100,

In [ ]:
result = fmin(
    fn=optimization_function,
    space=param_space,
    trials=trial,
    algo=tpe.suggest,
    max_evals=70
)

## HyperOpt Ends

In [ ]:
result

In [9]:
# params = {'alpha': 10, 'learning_rate': 0.1, 'max_depth': 8, 'scale_pos_weight' : scale_pos_weight}
# params = {'alpha': 10, 'learning_rate': 0.1, 'max_depth': 8, 'scale_pos_weight' : 11.646930462612744}
# params = {'alpha': 7, 'learning_rate': 0.14152368757577047, 'max_depth': 14, 'n_estimators': 359, 'scale_pos_weight' : scale_pos_weight}
# params = {'alpha': 8, 'learning_rate': 0.17908180749915578, 'max_depth': 10, 'n_estimators': 974}
# params = {
#     'alpha' : 7,
#     'learning_rate' : 0.12057882806776155,
#     'max_depth' : 16,
#     'n_estimators' : 775,
#     'gamma' : 5
# }

# params = {
#     'alpha' : 7,
#     'colsample_bytree' : 0.8,
#     'colsample_bynode' : 0.8,
#     'colsample_bylevel' : 0.8,
#     'learning_rate' : 0.01,
#     'max_depth' : 16,
#     'n_estimators' : 4000,
# #     'gamma' : 5,
#     'subsample' : 0.8,
#     'predictor' : 'gpu_predictor'
# }

params = {
    'alpha' : 7,
#     'colsample_bytree' : 0.8,
    'learning_rate' : 0.01,
    'max_depth' : 16,
    'n_estimators' : 3000,
#     'gamma' : 5,
    'subsample' : 0.8,
    'predictor' : 'gpu_predictor'
}

model = xgb.XGBClassifier(
        objective='binary:logistic',
        missing = -999,
        eval_metric=["error", "auc", "logloss"],
        random_state = 42,
        scale_pos_weight = get_scale_pos_weight(data['y_train']),
        tree_method = 'gpu_hist',
        gpu_id = 0,
        verbosity=1,
        **params)

eval_set = [(data['x_train'], data['y_train'].isFraud.ravel()), (data['x_val'], data['y_val'].isFraud.ravel())]

model.fit(data['x_train'], data['y_train'].isFraud.ravel(), eval_set=eval_set, verbose=True)

[0]	validation_0-error:0.07699	validation_0-auc:0.93584	validation_0-logloss:0.68668	validation_1-error:0.09138	validation_1-auc:0.85084	validation_1-logloss:0.68689
[1]	validation_0-error:0.07465	validation_0-auc:0.95693	validation_0-logloss:0.68031	validation_1-error:0.08833	validation_1-auc:0.87248	validation_1-logloss:0.68073
[2]	validation_0-error:0.06577	validation_0-auc:0.96611	validation_0-logloss:0.67395	validation_1-error:0.07944	validation_1-auc:0.88655	validation_1-logloss:0.67459
[3]	validation_0-error:0.06374	validation_0-auc:0.96815	validation_0-logloss:0.66791	validation_1-error:0.07627	validation_1-auc:0.89054	validation_1-logloss:0.66876
[4]	validation_0-error:0.06145	validation_0-auc:0.97053	validation_0-logloss:0.66177	validation_1-error:0.07427	validation_1-auc:0.89496	validation_1-logloss:0.66281
[5]	validation_0-error:0.06062	validation_0-auc:0.97145	validation_0-logloss:0.65584	validation_1-error:0.07332	validation_1-auc:0.89681	validation_1-logloss:0.65708
[6]	

[50]	validation_0-error:0.04185	validation_0-auc:0.98573	validation_0-logloss:0.45692	validation_1-error:0.05471	validation_1-auc:0.92027	validation_1-logloss:0.46541
[51]	validation_0-error:0.04176	validation_0-auc:0.98576	validation_0-logloss:0.45378	validation_1-error:0.05467	validation_1-auc:0.92017	validation_1-logloss:0.46241
[52]	validation_0-error:0.04154	validation_0-auc:0.98592	validation_0-logloss:0.45062	validation_1-error:0.05450	validation_1-auc:0.92039	validation_1-logloss:0.45938
[53]	validation_0-error:0.04164	validation_0-auc:0.98600	validation_0-logloss:0.44759	validation_1-error:0.05466	validation_1-auc:0.92032	validation_1-logloss:0.45647
[54]	validation_0-error:0.04142	validation_0-auc:0.98616	validation_0-logloss:0.44428	validation_1-error:0.05450	validation_1-auc:0.92054	validation_1-logloss:0.45331
[55]	validation_0-error:0.04144	validation_0-auc:0.98621	validation_0-logloss:0.44134	validation_1-error:0.05449	validation_1-auc:0.92054	validation_1-logloss:0.4504

[100]	validation_0-error:0.03505	validation_0-auc:0.99032	validation_0-logloss:0.33251	validation_1-error:0.04893	validation_1-auc:0.92516	validation_1-logloss:0.34656
[101]	validation_0-error:0.03498	validation_0-auc:0.99038	validation_0-logloss:0.33064	validation_1-error:0.04881	validation_1-auc:0.92514	validation_1-logloss:0.34477
[102]	validation_0-error:0.03482	validation_0-auc:0.99044	validation_0-logloss:0.32883	validation_1-error:0.04870	validation_1-auc:0.92524	validation_1-logloss:0.34305
[103]	validation_0-error:0.03474	validation_0-auc:0.99047	validation_0-logloss:0.32711	validation_1-error:0.04865	validation_1-auc:0.92523	validation_1-logloss:0.34140
[104]	validation_0-error:0.03469	validation_0-auc:0.99059	validation_0-logloss:0.32513	validation_1-error:0.04847	validation_1-auc:0.92541	validation_1-logloss:0.33953
[105]	validation_0-error:0.03455	validation_0-auc:0.99065	validation_0-logloss:0.32326	validation_1-error:0.04850	validation_1-auc:0.92540	validation_1-logloss:

[149]	validation_0-error:0.03064	validation_0-auc:0.99260	validation_0-logloss:0.26101	validation_1-error:0.04501	validation_1-auc:0.92798	validation_1-logloss:0.27875
[150]	validation_0-error:0.03054	validation_0-auc:0.99263	validation_0-logloss:0.25990	validation_1-error:0.04484	validation_1-auc:0.92804	validation_1-logloss:0.27770
[151]	validation_0-error:0.03045	validation_0-auc:0.99265	validation_0-logloss:0.25890	validation_1-error:0.04464	validation_1-auc:0.92809	validation_1-logloss:0.27677
[152]	validation_0-error:0.03043	validation_0-auc:0.99267	validation_0-logloss:0.25782	validation_1-error:0.04473	validation_1-auc:0.92812	validation_1-logloss:0.27575
[153]	validation_0-error:0.03034	validation_0-auc:0.99271	validation_0-logloss:0.25668	validation_1-error:0.04467	validation_1-auc:0.92820	validation_1-logloss:0.27468
[154]	validation_0-error:0.03030	validation_0-auc:0.99281	validation_0-logloss:0.25539	validation_1-error:0.04466	validation_1-auc:0.92832	validation_1-logloss:

[198]	validation_0-error:0.02721	validation_0-auc:0.99413	validation_0-logloss:0.21549	validation_1-error:0.04183	validation_1-auc:0.93084	validation_1-logloss:0.23587
[199]	validation_0-error:0.02715	validation_0-auc:0.99416	validation_0-logloss:0.21472	validation_1-error:0.04185	validation_1-auc:0.93091	validation_1-logloss:0.23515
[200]	validation_0-error:0.02709	validation_0-auc:0.99419	validation_0-logloss:0.21394	validation_1-error:0.04177	validation_1-auc:0.93100	validation_1-logloss:0.23442
[201]	validation_0-error:0.02700	validation_0-auc:0.99421	validation_0-logloss:0.21319	validation_1-error:0.04169	validation_1-auc:0.93104	validation_1-logloss:0.23371
[202]	validation_0-error:0.02698	validation_0-auc:0.99423	validation_0-logloss:0.21253	validation_1-error:0.04166	validation_1-auc:0.93107	validation_1-logloss:0.23308
[203]	validation_0-error:0.02692	validation_0-auc:0.99425	validation_0-logloss:0.21175	validation_1-error:0.04167	validation_1-auc:0.93115	validation_1-logloss:

...

[2844]	validation_0-error:0.00046	validation_0-auc:1.00000	validation_0-logloss:0.01399	validation_1-error:0.01568	validation_1-auc:0.95356	validation_1-logloss:0.05841
[2845]	validation_0-error:0.00046	validation_0-auc:1.00000	validation_0-logloss:0.01398	validation_1-error:0.01568	validation_1-auc:0.95356	validation_1-logloss:0.05841
[2846]	validation_0-error:0.00046	validation_0-auc:1.00000	validation_0-logloss:0.01398	validation_1-error:0.01568	validation_1-auc:0.95356	validation_1-logloss:0.05840
[2847]	validation_0-error:0.00046	validation_0-auc:1.00000	validation_0-logloss:0.01396	validation_1-error:0.01568	validation_1-auc:0.95356	validation_1-logloss:0.05839
[2848]	validation_0-error:0.00046	validation_0-auc:1.00000	validation_0-logloss:0.01396	validation_1-error:0.01568	validation_1-auc:0.95356	validation_1-logloss:0.05839
[2849]	validation_0-error:0.00046	validation_0-auc:1.00000	validation_0-logloss:0.01395	validation_1-error:0.01568	validation_1-auc:0.95356	validation_1-lo

[2893]	validation_0-error:0.00045	validation_0-auc:1.00000	validation_0-logloss:0.01361	validation_1-error:0.01565	validation_1-auc:0.95350	validation_1-logloss:0.05823
[2894]	validation_0-error:0.00045	validation_0-auc:1.00000	validation_0-logloss:0.01360	validation_1-error:0.01565	validation_1-auc:0.95351	validation_1-logloss:0.05822
[2895]	validation_0-error:0.00045	validation_0-auc:1.00000	validation_0-logloss:0.01359	validation_1-error:0.01565	validation_1-auc:0.95351	validation_1-logloss:0.05822
[2896]	validation_0-error:0.00045	validation_0-auc:1.00000	validation_0-logloss:0.01359	validation_1-error:0.01565	validation_1-auc:0.95351	validation_1-logloss:0.05821
[2897]	validation_0-error:0.00046	validation_0-auc:1.00000	validation_0-logloss:0.01358	validation_1-error:0.01567	validation_1-auc:0.95351	validation_1-logloss:0.05821
[2898]	validation_0-error:0.00045	validation_0-auc:1.00000	validation_0-logloss:0.01357	validation_1-error:0.01567	validation_1-auc:0.95352	validation_1-lo

[2942]	validation_0-error:0.00042	validation_0-auc:1.00000	validation_0-logloss:0.01328	validation_1-error:0.01560	validation_1-auc:0.95351	validation_1-logloss:0.05807
[2943]	validation_0-error:0.00042	validation_0-auc:1.00000	validation_0-logloss:0.01328	validation_1-error:0.01560	validation_1-auc:0.95351	validation_1-logloss:0.05807
[2944]	validation_0-error:0.00042	validation_0-auc:1.00000	validation_0-logloss:0.01327	validation_1-error:0.01560	validation_1-auc:0.95352	validation_1-logloss:0.05806
[2945]	validation_0-error:0.00042	validation_0-auc:1.00000	validation_0-logloss:0.01327	validation_1-error:0.01560	validation_1-auc:0.95352	validation_1-logloss:0.05806
[2946]	validation_0-error:0.00042	validation_0-auc:1.00000	validation_0-logloss:0.01326	validation_1-error:0.01560	validation_1-auc:0.95352	validation_1-logloss:0.05806
[2947]	validation_0-error:0.00042	validation_0-auc:1.00000	validation_0-logloss:0.01325	validation_1-error:0.01561	validation_1-auc:0.95352	validation_1-lo

[2991]	validation_0-error:0.00039	validation_0-auc:1.00000	validation_0-logloss:0.01298	validation_1-error:0.01559	validation_1-auc:0.95349	validation_1-logloss:0.05794
[2992]	validation_0-error:0.00039	validation_0-auc:1.00000	validation_0-logloss:0.01297	validation_1-error:0.01560	validation_1-auc:0.95349	validation_1-logloss:0.05793
[2993]	validation_0-error:0.00038	validation_0-auc:1.00000	validation_0-logloss:0.01297	validation_1-error:0.01560	validation_1-auc:0.95349	validation_1-logloss:0.05793
[2994]	validation_0-error:0.00038	validation_0-auc:1.00000	validation_0-logloss:0.01296	validation_1-error:0.01560	validation_1-auc:0.95349	validation_1-logloss:0.05793
[2995]	validation_0-error:0.00038	validation_0-auc:1.00000	validation_0-logloss:0.01296	validation_1-error:0.01559	validation_1-auc:0.95349	validation_1-logloss:0.05792
[2996]	validation_0-error:0.00038	validation_0-auc:1.00000	validation_0-logloss:0.01295	validation_1-error:0.01559	validation_1-auc:0.95349	validation_1-lo

XGBClassifier(alpha=7, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1,
              eval_metric=['error', 'auc', 'logloss'], gamma=0, gpu_id=0,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=16,
              min_child_weight=1, missing=-999,
              monotone_constraints='(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)',
              n_estimators=3000, n_jobs=0, num_parallel_tree=1,
              predictor='gpu_predictor', random_state=42, reg_alpha=7,
              reg_lambda=1, scale_pos_weight=27.43996147260274, subsample=0.8,
              tree_method='gpu_hist', validate_parameters=1, verbosity=1)

In [ ]:
results = model.evals_result()

In [ ]:
epochs = len(results['validation_0']['auc'])
x_axis = range(0, epochs)

# plot log loss
fig, ax = plt.subplots(figsize=(20, 20))
ax.plot(x_axis, results['validation_0']['auc'], label='Train')
ax.plot(x_axis, results['validation_1']['auc'], label='Test')
ax.legend()
plt.ylabel('AUC Score')
plt.title('XGBoost AUC Score')
plt.show()

# # plot classification error
fig, ax = plt.subplots(figsize=(20, 20))
ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
ax.plot(x_axis, results['validation_1']['logloss'], label='Test')
ax.legend()
plt.ylabel('Logloss')
plt.title('XGBoost Logloss')
plt.show()

# # plot classification error
fig, ax = plt.subplots(figsize=(20, 20))
ax.plot(x_axis, results['validation_0']['error'], label='Train')
ax.plot(x_axis, results['validation_1']['error'], label='Test')
ax.legend()
plt.ylabel('Classification Error')
plt.title('XGBoost Classification Error')
plt.show()

In [ ]:
model

In [ ]:
epochs = len(results['validation_0']['auc'])
x_axis = range(0, epochs)

# plot log loss
fig, ax = plt.subplots(figsize=(20, 20))
ax.plot(x_axis, results['validation_0']['auc'], label='Train')
ax.plot(x_axis, results['validation_1']['auc'], label='Test')
ax.legend()
plt.ylabel('AUC Score')
plt.title('XGBoost AUC Score')
plt.show()

# # plot classification error
fig, ax = plt.subplots(figsize=(20, 20))
ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
ax.plot(x_axis, results['validation_1']['logloss'], label='Test')
ax.legend()
plt.ylabel('Logloss')
plt.title('XGBoost Logloss')
plt.show()

# # plot classification error
fig, ax = plt.subplots(figsize=(20, 20))
ax.plot(x_axis, results['validation_0']['error'], label='Train')
ax.plot(x_axis, results['validation_1']['error'], label='Test')
ax.legend()
plt.ylabel('Classification Error')
plt.title('XGBoost Classification Error')
plt.show()

In [ ]:
y_pred_val = model.predict_proba(data['x_val'])[:, 1]
y_pred_train = model.predict_proba(data['x_train'])[:, 1]

print(f"ROC AUC Score\nTrain : {round(roc_auc_score( data['y_train'], y_pred_train ), 7)}\nTest : {round( roc_auc_score( data['y_val'], y_pred_val ), 7 )}")

In [ ]:
# model
from xgboost import plot_importance
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(40,60))
plot_importance(model, ax=ax)

In [40]:
import xgboost as xgb

# params = {'alpha': 10, 'learning_rate': 0.1, 'max_depth': 8, 'scale_pos_weight' : 11.646930462612744}
# params = {'alpha': 7, 'learning_rate': 0.14152368757577047, 'max_depth': 14, 'n_estimators': 359}
# learning_rate = 0.1, depth = 8, alpha = 10, boost_rounds = 100
# learning_rate = 0.1, depth = 8, alpha = 10, boost_rounds = 100, scale_pos_weight = 11.646930462612744

# params = {
#     'alpha' : 7,
#     'learning_rate' : 0.01,
#     'max_depth' : 16,
#     'n_estimators' : 2550,
# #     'gamma' : 5,
#     'subsample' : 0.8
# }

# params = {
#     'alpha' : 7,
#     'colsample_bytree' : 0.8,
#     'colsample_bynode' : 0.8,
#     'colsample_bylevel' : 0.8,
#     'learning_rate' : 0.01,
#     'max_depth' : 16,
#     'n_estimators' : 3000,
# #     'gamma' : 5,
#     'subsample' : 0.8,
#     'predictor' : 'gpu_predictor'
# }

params = {
    'alpha' : 7,
#     'colsample_bytree' : 0.8,
    'learning_rate' : 0.01,
    'max_depth' : 16,
    'n_estimators' : 3000,
#     'gamma' : 5,
    'subsample' : 0.8,
    'predictor' : 'gpu_predictor'
}


# params = {
#     'alpha' : 7,
#     'learning_rate' : 0.01,
#     'max_depth' : 16,
#     'n_estimators' : 3000,
# #     'gamma' : 5,
#     'subsample' : 0.8
# }

model = xgb.XGBClassifier(
        objective='binary:logistic',
        missing = -999,
        eval_metric='auc',
        random_state = 42,
        scale_pos_weight = get_scale_pos_weight(data['y']),
        tree_method = 'gpu_hist',
        gpu_id = 0,
        verbosity=1,
        **params)

model.fit(data['x'], data['y'].isFraud.ravel())

XGBClassifier(alpha=7, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0, gpu_id=0, importance_type='gain',
              interaction_constraints='', learning_rate=0.01, max_delta_step=0,
              max_depth=16, min_child_weight=1, missing=-999,
              monotone_constraints='(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)',
              n_estimators=3000, n_jobs=0, num_parallel_tree=1,
              predictor='gpu_predictor', random_state=42, reg_alpha=7,
              reg_lambda=1, scale_pos_weight=27.439200577895498, subsample=0.8,
              tree_method='gpu_hist', validate_parameters=1, verbosity=1)

In [41]:
model

XGBClassifier(alpha=7, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0, gpu_id=0, importance_type='gain',
              interaction_constraints='', learning_rate=0.01, max_delta_step=0,
              max_depth=16, min_child_weight=1, missing=-999,
              monotone_constraints='(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)',
              n_estimators=3000, n_jobs=0, num_parallel_tree=1,
              predictor='gpu_predictor', random_state=42, reg_alpha=7,
              reg_lambda=1, scale_pos_weight=27.439200577895498, subsample=0.8,
              tree_method='gpu_hist', validate_parameters=1, verbosity=1)

In [42]:
preds_proba = model.predict_proba(data['test'])
preds_proba

array([[9.9116510e-01, 8.8349022e-03],
       [9.8831564e-01, 1.1684372e-02],
       [9.9923545e-01, 7.6456118e-04],
       ...,
       [9.9040240e-01, 9.5976060e-03],
       [9.9765503e-01, 2.3449813e-03],
       [9.9620336e-01, 3.7966301e-03]], dtype=float32)

In [43]:
preds_proba = preds_proba[:, 1]
preds_proba

array([0.0088349 , 0.01168437, 0.00076456, ..., 0.00959761, 0.00234498,
       0.00379663], dtype=float32)

In [50]:
preds_proba

array([0.0088349 , 0.01168437, 0.00076456, ..., 0.00959761, 0.00234498,
       0.00379663], dtype=float32)

In [70]:
pd.DataFrame( { 'val' : preds_proba[preds_proba > 0.5] } ).sort_values('val')

,val
4760,0.500015
3692,0.500050
6064,0.500302
383,0.500437
6195,0.500959
...,...
1537,0.999972
4978,0.999974
1017,0.999977
4605,0.999978


In [79]:
len( preds_proba[preds_proba > 0.8] )

5182

In [85]:
mod_thres = [ 1 if x > 0.8 else 0 for x in preds_proba]

In [86]:
len( mod_thres )

236216

In [87]:
pd.DataFrame(mod_thres).value_counts()

0    231034
1      5182
dtype: int64

In [18]:
pd.DataFrame(preds).value_counts()

0    229396
1      6820
dtype: int64

In [90]:
mod_thres

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [88]:
lol = data['y']

In [89]:
lol

,isFraud
TransactionID,
2987002,0
2990409,0
2990410,0
2990411,0
2990412,0
...,...
3575688,0
3575686,0
3575685,0


In [91]:
sample = pd.read_csv('../input/frauddetect/sample_submission.csv')
sample.isFraud = mod_thres
sample

,TransactionID,isFraud
0,2987000,0
1,2987001,0
2,2987004,0
3,2987007,0
4,2987009,0
...,...,...
236211,3577532,0
236212,3577533,0
236213,3577534,0
236214,3577535,0


In [92]:
y_pseudo = sample.set_index('TransactionID')
y_pseudo

,isFraud
TransactionID,
2987000,0
2987001,0
2987004,0
2987007,0
2987009,0
...,...
3577532,0
3577533,0
3577534,0


In [93]:
data['y']

,isFraud
TransactionID,
2987002,0
2990409,0
2990410,0
2990411,0
2990412,0
...,...
3575688,0
3575686,0
3575685,0


In [94]:
pseudo_labels = pd.concat( [data['y'], y_pseudo] )
pseudo_labels

,isFraud
TransactionID,
2987002,0
2990409,0
2990410,0
2990411,0
2990412,0
...,...
3577532,0
3577533,0
3577534,0


In [95]:
pseudo_x = pd.concat( [data['x'], data['test']])
pseudo_x

,encoder__x0_C,encoder__x0_H,encoder__x0_R,encoder__x0_S,encoder__x0_W,encoder__x1_123.0,encoder__x1_126.0,encoder__x1_143.0,encoder__x1_181.0,encoder__x1_184.0,...,id_01,id_02,id_05,id_06,id_11,id_13,id_14,id_17,id_19,id_20
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,22608.0,1.0,-11.0,100.0,0.0,-6.0,166.0,410.0,146.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236211,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
236212,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
236213,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-45.0,339406.0,-10.0,-100.0,100.0,2.0,-999.0,225.0,417.0,595.0
236214,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [96]:
import xgboost as xgb

params = {
    'alpha' : 7,
    'learning_rate' : 0.01,
    'max_depth' : 16,
    'n_estimators' : 3000,
    'subsample' : 0.8,
    'predictor' : 'gpu_predictor'
}

model = xgb.XGBClassifier(
        objective='binary:logistic',
        missing = -999,
        eval_metric='auc',
        random_state = 42,
        scale_pos_weight = get_scale_pos_weight(data['y']),
        tree_method = 'gpu_hist',
        gpu_id = 0,
        verbosity=1,
        **params)

model.fit(pseudo_x, pseudo_labels.isFraud.ravel())

XGBClassifier(alpha=7, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0, gpu_id=0, importance_type='gain',
              interaction_constraints='', learning_rate=0.01, max_delta_step=0,
              max_depth=16, min_child_weight=1, missing=-999,
              monotone_constraints='(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)',
              n_estimators=3000, n_jobs=0, num_parallel_tree=1,
              predictor='gpu_predictor', random_state=42, reg_alpha=7,
              reg_lambda=1, scale_pos_weight=27.439200577895498, subsample=0.8,
              tree_method='gpu_hist', validate_parameters=1, verbosity=1)

In [97]:
model

XGBClassifier(alpha=7, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0, gpu_id=0, importance_type='gain',
              interaction_constraints='', learning_rate=0.01, max_delta_step=0,
              max_depth=16, min_child_weight=1, missing=-999,
              monotone_constraints='(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)',
              n_estimators=3000, n_jobs=0, num_parallel_tree=1,
              predictor='gpu_predictor', random_state=42, reg_alpha=7,
              reg_lambda=1, scale_pos_weight=27.439200577895498, subsample=0.8,
              tree_method='gpu_hist', validate_parameters=1, verbosity=1)

In [ ]:
y_pred_train = model.predict_proba(data['x'])[:, 1]
y_pred_train

In [ ]:
print(f"ROC AUC Score\nTrain : {round(roc_auc_score( data['y'], y_pred_train ), 7)}")

In [98]:
y_pred_test = model.predict_proba(data['test'])[:, 1]
y_pred_test

array([0.00537626, 0.00626695, 0.00037999, ..., 0.0049948 , 0.00129664,
       0.00218144], dtype=float32)

In [99]:
submit(y_pred_test, 38)